In [1]:
%cd /home/tae898/repos/erc

/home/tae898/repos/erc


In [2]:
ROOT_DIR = './multimodal-datasets/'
model_checkpoint = 'results/MELD/roberta-large/2021-05-22-19-51-21/'
# model_checkpoint = 'results/IEMOCAP/roberta-large/2021-05-21-07-58-04/3/checkpoint-4780/'
DATASET = model_checkpoint.split('/')[1]


import yaml

def read_json(path):
    with open(path, 'r') as stream:
        foo = json.load(stream)
    return foo

def read_yaml(path):
    with open(path, 'r') as stream:
        foo = yaml.load(stream)
    return foo


kwargs_path = f"{'/'.join(model_checkpoint.split('/')[:-3])}/kwargs.yaml"    
kwargs = read_yaml(kwargs_path)
kwargs

/home/tae898/.virtualenvs/dev-python3.7/lib/python3.7/site-packages/ipykernel_launcher.py:16: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  app.launch_new_instance()


{'ADD_BOU_EOU': True,
 'ADD_SPEAKER_TOKENS': True,
 'BATCH_SIZE': 4,
 'DATASET': 'MELD',
 'HP_N_TRIALS': 5,
 'HP_ONLY_UPTO': 103,
 'NUM_TRAIN_EPOCHS': 5,
 'REPLACE_NAMES_IN_UTTERANCES': False,
 'SEEDS': [0, 1, 2, 3, 4],
 'SPEAKER_SPLITS': ['train', 'val', 'test'],
 'WARMUP_RATIO': 0.2,
 'WEIGHT_DECAY': 0.01,
 'model_checkpoint': 'roberta-large',
 'num_future_utterances': 1000,
 'num_past_utterances': 1000}

In [4]:
from transformers import RobertaTokenizerFast, RobertaForSequenceClassification
from utils import get_num_classes, ErcTextDataset
import numpy as np
import matplotlib.pyplot as plt

NUM_CLASSES = get_num_classes(DATASET)
tokenizer = RobertaTokenizerFast.from_pretrained(
    model_checkpoint)

model = RobertaForSequenceClassification.from_pretrained(
    model_checkpoint, num_labels=NUM_CLASSES)

model.eval()
model.cpu()

SEED = int(model_checkpoint.split('/')[-3])
print(SEED)

ds_train = ErcTextDataset(DATASET=DATASET, SPLIT='train',
                          num_past_utterances=kwargs['num_past_utterances'], num_future_utterances=kwargs['num_future_utterances'],
                          model_checkpoint=f"{'/'.join(model_checkpoint.split('/')[:-3])}/tokenizer",
                          ADD_BOU_EOU=kwargs['ADD_BOU_EOU'], ADD_SPEAKER_TOKENS=f"{'/'.join(model_checkpoint.split('/')[:-3])}/tokenizer/added_tokens.json",
                          REPLACE_NAMES_IN_UTTERANCES=kwargs['REPLACE_NAMES_IN_UTTERANCES'],
                          ROOT_DIR=ROOT_DIR, SEED=SEED)

ds_val = ErcTextDataset(DATASET=DATASET, SPLIT='val',
                          num_past_utterances=kwargs['num_past_utterances'], num_future_utterances=kwargs['num_future_utterances'],
                          model_checkpoint=f"{'/'.join(model_checkpoint.split('/')[:-3])}/tokenizer",
                          ADD_BOU_EOU=kwargs['ADD_BOU_EOU'], ADD_SPEAKER_TOKENS=f"{'/'.join(model_checkpoint.split('/')[:-3])}/tokenizer/added_tokens.json",
                          REPLACE_NAMES_IN_UTTERANCES=kwargs['REPLACE_NAMES_IN_UTTERANCES'],
                          ROOT_DIR=ROOT_DIR, SEED=SEED)

ds_test = ErcTextDataset(DATASET=DATASET, SPLIT='test',
                          num_past_utterances=kwargs['num_past_utterances'], num_future_utterances=kwargs['num_future_utterances'],
                          model_checkpoint=f"{'/'.join(model_checkpoint.split('/')[:-3])}/tokenizer",
                          ADD_BOU_EOU=kwargs['ADD_BOU_EOU'], ADD_SPEAKER_TOKENS=f"{'/'.join(model_checkpoint.split('/')[:-3])}/tokenizer/added_tokens.json",
                          REPLACE_NAMES_IN_UTTERANCES=kwargs['REPLACE_NAMES_IN_UTTERANCES'],
                          ROOT_DIR=ROOT_DIR, SEED=SEED)

0


2021-05-21 14:02:02.488 INFO utils - _load_utterance_ordered: every text file exists fine!
2021-05-21 14:02:02.489 INFO utils - _string2tokens: converting utterances into tokens ...
2021-05-21 14:02:02.491 INFO utils - _string2tokens: creating input utterance data ... 
100%|██████████| 1038/1038 [00:47<00:00, 22.07it/s]
2021-05-21 14:02:50.429 INFO utils - _create_input: number of truncated utterances: 68
2021-05-21 14:02:50.483 INFO utils - _load_utterance_ordered: every text file exists fine!
2021-05-21 14:02:50.484 INFO utils - _string2tokens: converting utterances into tokens ...
2021-05-21 14:02:50.485 INFO utils - _string2tokens: creating input utterance data ... 
100%|██████████| 114/114 [00:03<00:00, 29.70it/s]
2021-05-21 14:02:55.223 INFO utils - _create_input: number of truncated utterances: 0
2021-05-21 14:02:55.308 INFO utils - _load_utterance_ordered: every text file exists fine!
2021-05-21 14:02:55.309 INFO utils - _string2tokens: converting utterances into tokens ...
202

In [10]:
SPLIT = 'val'
if SPLIT == 'train':
    ds = ds_train
elif SPLIT == 'val':
    ds = ds_val
else:
    ds = ds_test

idx = np.random.randint(0, len(ds))
uttid = ds.uttids[idx]
diaid = None

for diaid_, uttids_ in ds.utterance_ordered.items():
    for uttid_ in uttids_:
        if uttid_ == uttid:
            diaid = diaid_    

from glob import glob
import json
from pprint import pprint

utts = {foo.split('/')[-1].split('.json')[0]: f"({read_json(foo)['Emotion']}) " + f"<{read_json(foo)['Speaker']}>" + read_json(foo)['Utterance'] for foo in glob(f'./multimodal-datasets/{DATASET}/raw-texts/{SPLIT}/{diaid}_*.json')}
# utts_keys = [foo[0] for foo in sorted([(foo, int(foo.split('utt')[1])) for foo in list(utts.keys())], key=lambda x:x[1])]
utts = {key: utts[key] for key in ds.utterance_ordered[diaid]}
utt = utts[uttid]
input_ids = ds[idx]['input_ids']

print(f'idx: {idx}')
print()
print(f"diaid: {diaid} \nuttid: {uttid}")
print()
print(input_ids)
print()
print(tokenizer.decode(input_ids))
print()
print(utt)
print()
for key, val in utts.items():
    pprint(f"{key}, {val}")


idx: 1033

diaid: dia87 
uttid: dia87_utt0

[0, 2, 2, 50497, 7516, 734, 15983, 6, 2051, 4, 2, 2, 50265, 50497, 1708, 734, 364, 13492, 734, 50266, 50265, 50497, 100, 95, 33, 65, 864, 13, 47, 6, 1437, 1437, 1437, 364, 13492, 734, 50266, 50265, 50497, 1779, 52, 4205, 197, 52, 1656, 6, 50, 422, 6, 50, 3349, 2389, 6, 1437, 1437, 1437, 50, 24808, 734, 50266, 50265, 50301, 22174, 24, 6, 912, 24, 328, 91, 1431, 2635, 53, 37, 429, 32734, 1769, 328, 5148, 6, 1437, 1437, 1437, 905, 18, 213, 12846, 50266, 2]

<s></s></s><Ross>Oh... ok, fine.</s></s><u><Ross>But... ehm...</u><u><Ross>I just have one question for you,    ehm...</u><u><Ross>When we exit should we walk, or run, or prance,    or stroll...</u><u><Charlie>Stop it, stop it! He talks slow but he might pee fast! Ok,    let's go!!</u></s>

(neutral) <Ross>Oh... ok, fine.

'dia87_utt0, (neutral) <Ross>Oh... ok, fine.'
'dia87_utt1, (neutral) <Ross>But... ehm...'
'dia87_utt2, (neutral) <Ross>I just have one question for you,    ehm...'
('dia87_

In [11]:
# utt = 'I am a stupid fuck, Im Ross'
print(utt)
print(tokenizer(utt))

for token, token_id in ds.added_tokens.items():
    if token in ['<u>', '</u>']:
        continue
    token_stripped = token.split('<')[-1].split('>')[0]
    
    utt = utt.replace(token_stripped, token)
    
print()

print(utt)
print(tokenizer(utt))

(neutral) <Ross>Oh... ok, fine.
{'input_ids': [0, 1640, 12516, 43, 1437, 50497, 7516, 734, 15983, 6, 2051, 4, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

(neutral) <<Ross>>Oh... ok, fine.
{'input_ids': [0, 1640, 12516, 43, 28696, 50497, 15698, 7516, 734, 15983, 6, 2051, 4, 2], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
